# Text Generation

This will be an example of using the KERAS to do text generation at a character level


In [1]:
# importing all of the libraries

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
import nltk
nltk.download("punkt")

# grabbing the text and lowering each one
text=(open("/content/drive/MyDrive/NLP Notes/Mix.txt").read())

text=text.lower()

# creating a dictonary of each alpha numeric text occurence in the soonet: 
words = sorted(list(set(text)))
# these are feature dictonaries, they hold the index location for each of them
n_to_words = {n:char for n, char in enumerate(words)}
words_to_n = {char:n for n, char in enumerate(words)}

# showing what these look like when you
for key in n_to_words:
    print(key, n_to_words[key])
for key in words_to_n:
    print(key,words_to_n[key])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
0 

1  
2 !
3 "
4 $
5 &
6 '
7 (
8 )
9 ,
10 -
11 .
12 0
13 1
14 2
15 3
16 4
17 5
18 7
19 8
20 9
21 :
22 ;
23 ?
24 [
25 ]
26 a
27 b
28 c
29 d
30 e
31 f
32 g
33 h
34 i
35 j
36 k
37 l
38 m
39 n
40 o
41 p
42 q
43 r
44 s
45 t
46 u
47 v
48 w
49 x
50 y
51 z
52 ¡
53 ç
54 î
55 ó
56 ’

 0
  1
! 2
" 3
$ 4
& 5
' 6
( 7
) 8
, 9
- 10
. 11
0 12
1 13
2 14
3 15
4 16
5 17
7 18
8 19
9 20
: 21
; 22
? 23
[ 24
] 25
a 26
b 27
c 28
d 29
e 30
f 31
g 32
h 33
i 34
j 35
k 36
l 37
m 38
n 39
o 40
p 41
q 42
r 43
s 44
t 45
u 46
v 47
w 48
x 49
y 50
z 51
¡ 52
ç 53
î 54
ó 55
’ 56




#Understanding how the data is being set up:

The next step is to prepare how the data will be fed into the LSTM and how you to hold the output.
--
**First** -- we establish a context that we want the model to look at. This is our look up length, basically its how many examples of letters do we have before to predict the next letter
-- 
**Second** -- we make a reference for every single character in the text, THESE are huge arrays but provide the LSTM with 100 examples for each character till the end of our document
-- 
**Last** -- we provide the true values in the target array, i.e what the letter should be given the last 100 examples it had
--

In [2]:
# setting up the arrays to hold data
train_array = []
target_array = []

# getting the length of the text document
length = len(text)

# Setting up how many characters we want the model to look up for context
look_up_length = 50

# creating a look-up table so that the model understands the context around the character its looking at
for i in range(0, length-look_up_length, 1):
    # creating the squence of characters for each context window
    sequence = text[i:i+look_up_length]
    # creating a label to reference each character in the sequence
    labels = text[i+look_up_length]
    # appending the sqeunce of characters to the training array
    train_array.append([words_to_n[char]for char in sequence])
    # adding the label of what character should come next for each sequence
    target_array.append(words_to_n[labels])


The next step is to transform the data so that our model can actually understand it

In [3]:
# making the training array into 3-D
modded_training = np.reshape(train_array, (len(train_array), look_up_length, 1))
# normalizing by the modifed training data
final_mod_training = modded_training / float(len(words)) 
# Modding the target array to remove relationships of of distance within the array 
final_mod_target = np_utils.to_categorical(target_array)

# giving a idea of what these look like
print(final_mod_target[1] )
print(final_mod_training[1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0.68421053]
 [0.49122807]
 [0.52631579]
 [0.01754386]
 [0.80701754]
 [0.71929825]
 [0.70175439]
 [0.68421053]
 [0.01754386]
 [0.45614035]
 [0.01754386]
 [0.78947368]
 [0.59649123]
 [0.66666667]
 [0.52631579]
 [0.15789474]
 [0.01754386]
 [0.59649123]
 [0.01754386]
 [0.77192982]
 [0.78947368]
 [0.59649123]
 [0.64912281]
 [0.64912281]
 [0.01754386]
 [0.47368421]
 [0.52631579]
 [0.64912281]
 [0.59649123]
 [0.52631579]
 [0.8245614 ]
 [0.52631579]
 [0.50877193]
 [0.01754386]
 [0.59649123]
 [0.68421053]
 [0.01754386]
 [0.66666667]
 [0.87719298]
 [0.77192982]
 [0.52631579]
 [0.64912281]
 [0.54385965]
 [0.        ]
 [0.59649123]
 [0.01754386]
 [0.84210526]
 [0.45614035]
 [0.77192982]
 [0.01754386]]


Experimental model

In [4]:
from tensorflow import keras
# defining the model as sequential
model = Sequential()
# adding the inputs from the training data to a LSTM model
model.add(LSTM(500, input_shape=(final_mod_training.shape[1], final_mod_training.shape[2]), return_sequences=True))
# adding a drop out layer
model.add(Dropout(0.3))
# adding our second LSTM model
model.add(LSTM(500, return_sequences = True))
# adding a second drop out layer
model.add(Dropout(0.3))
# adding  a third layer
model.add(LSTM(500))
# adding a third drop out layer
model.add(Dropout(0.3))
# adding a dense layer
model.add(Dense(final_mod_target.shape[1], activation='softmax'))
# compling the model
model.compile(loss='categorical_crossentropy', optimizer="adam")

Viewing a summary of the model

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 500)           1004000   
_________________________________________________________________
dropout (Dropout)            (None, 50, 500)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 500)           2002000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 500)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 500)               2002000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense (Dense)                (None, 57)                2

Traing the model

In [ ]:
model.fit(final_mod_training,final_mod_target, epochs=100, batch_size=64)

Epoch 1/100
3391/3391 [==============================] - 141s 41ms/step - loss: 2.7531
Epoch 2/100
3391/3391 [==============================] - 146s 43ms/step - loss: 2.3921
Epoch 3/100
3391/3391 [==============================] - 149s 44ms/step - loss: 2.1463
Epoch 4/100
3391/3391 [==============================] - 149s 44ms/step - loss: 1.9975
Epoch 5/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.8914
Epoch 6/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.8113
Epoch 7/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.7432
Epoch 8/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.6805
Epoch 9/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.6247
Epoch 10/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.5729
Epoch 11/100
3391/3391 [==============================] - 148s 44ms/step - loss: 1.5265
Epoch 12/100
3391/3391 [=================

Original Model

In [ ]:
#@title
from tensorflow import keras
# defining the model as sequential
model = Sequential()
# adding the inputs from the training data to a LSTM model
model.add(LSTM(500, input_shape=(final_mod_training.shape[1], final_mod_training.shape[2]), return_sequences=True))
# adding a drop out layer
model.add(Dropout(0.4))
# adding our second LSTM model
model.add(LSTM(500, return_sequences = True))
# adding a second drop out layer
model.add(Dropout(0.4))
# adding  a third layer
model.add(LSTM(500))
# adding a third drop out layer
model.add(Dropout(0.4))
# adding a dense layer
model.add(Dense(final_mod_target.shape[1], activation='softmax'))
# setting up the optimizer
opt = keras.optimizers.Adam(learning_rate=0.0124)
# compling the model
model.compile(loss='categorical_crossentropy', optimizer="adam")

# CODE BELOW NEEDS SOME WORK SON

In [6]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/text_generator_giganticv2_65epocs.h5')

In [11]:
import random as rnd
# having the system pick a random training sentence
input_string  = rnd.choice(train_array) 
# converting the string into something the computer likes
full_string = [n_to_words[value] for value in input_string]
# the phrase the model is going off of
lyrics = ''
for i in full_string:
    lyrics += i
print(lyrics)

es, lotta folds, i'm still tryin' to just get back


# Getting it generate!

The model really does need some work, but I think the main thing
is that the dataset just needs to be a lot bigger to get a better generation
right now there is only about 70 or 80 songs in the test file, but I actually like the jank-ness of some the lyrics it generates

In [12]:
# Setting it equal to the input string generated above
string_mapped=input_string
full_string = [n_to_words[value] for value in string_mapped]

# Giving the string the values the model can use
lyrics = ''
for i in full_string:
    lyrics += i
print(lyrics)


# generating characters
for i in range(250):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(words))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_words[value] for value in string_mapped]
    full_string.append(n_to_words[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

# Printing the outcome of the models predictions
txt=""
for char in full_string:
    txt = txt+char
print(txt)


es, lotta folds, i'm still tryin' to just get back
es, lotta folds, i'm still tryin' to just get back up

i don't know what that means, but it's something to me that
you make me feel better
i don't gotta put you that you know you gotta lake you cringe

i got the day i go insane
that'll be the day, today might be the day i go insane
the day i go insa
